In [1]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, value):
        self.value = value

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

In [2]:
# Representation of 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

这样做的问题是对于每个表达式,每次都要重新定义一遍,有没有一种更通用的方式让它支持所有的数字和操作符呢？  
这里我们使用访问者模式。

In [3]:
class NodeVisitor:
    def visit(self, node):
        methname = "visit_" + type(node).__name__
        meth    = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise NotImplementedError("no {} method".format('visit_'+type(node).__name__))

为了使用这个类,可以定义一个类继承它并且实现各种 visit_Name() 方法,其中Name 是 node 类型。

In [4]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    
    def visit_Negate(self, node):
        return -self.value

In [5]:
e = Evaluator()
e.visit(t4)

0.6

访问者模式一个缺点就是它严重依赖递归,如果数据结构嵌套层次太深可能会有
问题,有时候会超过 Python 的递归深度限制

In [10]:
import types

class NodeVisitor:
    def visit(self, node):
        stack = [node]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                elif instance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except:
                stack.pop()

        return last_result
    
    def _visit(self, node):
        meth_name = 'visit_' + type(node).__name__
        meth = getattr(self, meth_name, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise NotImplementedError("no {} method".format(types(node).__name__))

print(1234)

1234


In [11]:
class Evaluator2(NodeVisitor):
    def visit_Number(self, node):
        yield node.value
    
    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)
    
    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)
    
    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)
    
    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)
    
    def visit_Negate(self, node):
        yield -(yield node.operand)

e2 = Evaluator2()
e2.visit(t4)